# Import librarys

In [1]:
from keras.models import load_model
from keras.metrics import MeanSquaredError
from sklearn.base import BaseEstimator
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import keras
from keras.callbacks import History
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import keras
from scikeras.wrappers import KerasRegressor
from keras.layers import Input
from keras.optimizers import RMSprop
import yfinance as yf
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense
from tensorflow.keras.losses import MeanSquaredError
from sklearn.base import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV

KeyboardInterrupt: 

# CSV einlesen

In [ ]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # Schreiben der Spaltenüberschriften
        f.write('Date,Open,High,Low,Close,Volume\n')
        
        for symbol in stock_symbols:
            # Herunterladen der Aktiendaten für das Symbol und den angegebenen Zeitraum
            stock_data = yf.download(symbol, start=start_date, end=end_date)
            stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Auswahl der gewünschten Spalten
            stock_data.to_csv(f, header=False)  # Schreiben der Daten in die Datei


In [ ]:
# Benutzerdefinierten Zeitraum und Symbole eingeben
stock_symbols = ['ADS.DE']
start_date = '2014-01-01'
end_date = '2018-12-31'
output_file = 'stock_data.csv'

# Herunterladen und Speichern der Aktiendaten
download_stock_data(stock_symbols, start_date, end_date, output_file)

# CSV-Datei nach Leerzeilen durchsuchen und entfernen
with open(output_file, 'r') as file:
    lines = file.readlines()

# Filtern der Leerzeilen aus den Zeilen
lines = [line.strip() for line in lines if line.strip()]

# Überschreiben der Datei mit den bereinigten Zeilen
with open(output_file, 'w') as file:
    file.write('\n'.join(lines))

In [ ]:
# load CSV
df = pd.read_csv('stock_data.csv')

# transform date to index
df.set_index('Date', inplace=True)

# Show first rows
print(df.head())
print(len(df))

In [ ]:
# Überprüfen von fehlenden Werten vor der Behandlung
missing_values_before = df.isnull().values.any()

# Fehlende Werte für alle Spalten behandeln
for column in df.columns:
    missing_values = df[column].isnull()
    df.loc[missing_values, column] = (df[column].shift() + df[column].shift(-1)) / 2

# Überprüfen von fehlenden Werten nach der Behandlung
missing_values_after = df.isnull().values.any()

# Ausgabe, ob fehlende Werte vor und nach der Behandlung vorhanden sind
print("Fehlende Werte vor der Behandlung gefunden:", missing_values_before)
print("Fehlende Werte nach der Behandlung gefunden:", missing_values_after)

In [ ]:
# show first rows
print(df.head())

# Data normalization

In [ ]:
# Kopieren des DataFrames und Entfernen der Spalte "Date"
nf = df.copy()

# Entfernen des Indexnamens
nf.index.name = None

# Normalisierung der Daten außer dem Datum (Index)
scaler = MinMaxScaler()
nf_normalized = scaler.fit_transform(nf)

# Erstellen eines neuen DataFrames mit normalisierten Daten und dem ursprünglichen Index
nf = pd.DataFrame(nf_normalized, columns=nf.columns, index=nf.index)

# Anzeigen der normalisierten Daten
print(nf.head())

# Anzeigen der Länge des Datensatzes
print("Länge des Datensatzes:", len(nf))

In [ ]:
# save scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# read predicting sequence

In [ ]:
# Benutzerdefinierten Zeitraum und Symbole eingeben
stock_symbols = ['BNR.VI']
start_date = '2018-10-01'
end_date = '2018-12-31'
output_file = 'stock_data_pre.csv'

# Herunterladen und Speichern der Aktiendaten
download_stock_data(stock_symbols, start_date, end_date, output_file)

# CSV-Datei nach Leerzeilen durchsuchen und entfernen
with open(output_file, 'r') as file:
    lines = file.readlines()

# Filtern der Leerzeilen aus den Zeilen
lines = [line.strip() for line in lines if line.strip()]

# Überschreiben der Datei mit den bereinigten Zeilen
with open(output_file, 'w') as file:
    file.write('\n'.join(lines))

In [ ]:
# load CSV
dfPre = pd.read_csv('stock_data_pre.csv')

# Drop empty rows
dfPre.dropna(inplace=True)

# transform date to index
dfPre.set_index('Date', inplace=True)

# Check the number of rows
num_rows = dfPre.shape[0]

# If more than 60 rows, keep the last 60 rows
if num_rows > 60:
    dfPre = dfPre.tail(60)

# Show first rows
print(dfPre.head())

In [ ]:
# Überprüfen von fehlenden Werten vor der Behandlung
missing_values_beforePre = dfPre.isnull().values.any()

# Fehlende Werte für alle Spalten behandeln
for column in df.columns:
    missing_valuesPre = dfPre[column].isnull()
    dfPre.loc[missing_valuesPre, column] = (dfPre[column].shift() + dfPre[column].shift(-1)) / 2

# Überprüfen von fehlenden Werten nach der Behandlung
missing_values_afterPre = dfPre.isnull().values.any()

# Ausgabe, ob fehlende Werte vor und nach der Behandlung vorhanden sind
print("Fehlende Werte vor der Behandlung gefunden:", missing_values_beforePre)
print("Fehlende Werte nach der Behandlung gefunden:", missing_values_afterPre)

# show lenth of data set
print(len(dfPre))

# Data normalization prediction data

In [ ]:
# Kopieren des DataFrames und Entfernen der Spalte "Date"
nfPre = dfPre.copy()

# Entfernen des Indexnamens
nfPre.index.name = None

# Normalisierung der Daten außer dem Datum (Index)
nfPre_normalized = scaler.fit_transform(nfPre)

# Erstellen eines neuen DataFrames mit normalisierten Daten und dem ursprünglichen Index
nfPre = pd.DataFrame(nfPre_normalized, columns=nfPre.columns, index=nfPre.index)

# Convert DataFrame to NumPy array
nfPre = nfPre.to_numpy()

# Anzeigen der Länge des Datensatzes
print("Länge des Datensatzes:", len(nfPre))


# Create Sequences

In [ ]:
# Function to create sequenences for LSTM model
def create_sequences(data_input, seq_length):
    data = data_input
    X, y = [], []
    for i in range(len(data) - seq_length - 20):
        X.append(data[i:i+seq_length])
        y.append(data.iloc[i+seq_length+20]) # acess to Close-Value of the 20th row
        #if (i < 10):
            #print(X)
            #print("Ausgabe y:")
            #print(y)
    return np.array(X), np.array(y)

In [ ]:
# create sequence
sequence_length = 60
X, y = create_sequences(nf, sequence_length)

# Split the data in Training, Test and validation

In [ ]:
# Split data in training and test (80% training, 20% test)
train_data, test_data = train_test_split(nf, test_size=0.2, shuffle=False)

# Split data in training and validation (80% training, 20% validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, shuffle=False)

# Aufteilen der Trainingsdaten in Sequenzen
X_train, y_train = create_sequences(train_data, sequence_length)

# Aufteilen der Validierungsdaten in Sequenzen
X_val, y_val = create_sequences(val_data, sequence_length)

# Aufteilen der Testdaten in Sequenzen
X_test, y_test = create_sequences(test_data, sequence_length)

# show training, validation and test sets
print("Trainingsdaten:", len(train_data))
print("Validierungsdaten:", len(val_data))
print("Testdaten:", len(test_data))

In [ ]:
# Define train_size, val_size and test_size
train_size = len(train_data)
val_size = len(val_data)
test_size = len(test_data)

# Extract features and target variables for training, validation, and test sets
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print(X_train.shape)
print(y_train.shape)

In [ ]:
# Konvertieren von Trainings- und Validierungsdaten in TensorFlow-Tensoren
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype=tf.float32)

print(type(X_train_tensor))
print(type(y_train_tensor))

# Define model

In [ ]:
param_dist = {
    'units': [25, 50, 100, 300, 400, 500, 800],
    'dropout_rate': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
    'batch_size': [16, 32, 64, 128, 256],
    'epochs': [10, 20, 30, 40, 50, 80, 100],
    'optimizer': ['adam', 'rmsprop'],
    'learning_rate': [0.001, 0.003, 0.005, 0.008, 0.01],
}

In [ ]:
class LSTMModel(BaseEstimator):
    def __init__(self, batch_size=32, optimizer='adam', units=50, dropout_rate=0.2, epochs=10, learning_rate=0.001):
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.units = units
        self.dropout_rate = dropout_rate
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.model = self.create_lstm_model()  # Hier das Modell erstellen
        
    def create_lstm_model(self):
        if self.optimizer == 'adam':
            optimizer = Adam(learning_rate=self.learning_rate)
        elif self.optimizer == 'rmsprop':
            optimizer = RMSprop(learning_rate=self.learning_rate)
        else:
            raise ValueError("Unrecognized optimizer name.")
        input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
        lstm_layer1 = LSTM(units=self.units, return_sequences=True)(input_layer)
        dropout_layer1 = Dropout(self.dropout_rate)(lstm_layer1)
        lstm_layer2 = LSTM(units=self.units, return_sequences=False)(dropout_layer1)
        dropout_layer2 = Dropout(self.dropout_rate)(lstm_layer2)
        output_layer = Dense(units=5)(dropout_layer2)
    
        model = Model(inputs=input_layer, outputs=output_layer)
    
        model.compile(optimizer=optimizer, loss='mse', metrics=[MeanSquaredError()])
    
        return model
        
    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size)
        
    def predict(self, X):
        return self.model.predict(X)

In [ ]:
lstm_model = LSTMModel()  # Hier wird eine Instanz von LSTMModel erstellt
random_search = RandomizedSearchCV(estimator=lstm_model,
                                   param_distributions=param_dist,
                                   n_iter=10,
                                   cv=3,
                                   scoring='neg_mean_squared_error',
                                   verbose=2,
                                   random_state=42)

# Trainieren Sie das Modell mit den besten Hyperparametern
random_search.fit(X_train, y_train)

In [ ]:
# Beste Hyperparameter ausgeben
print("Beste Hyperparameter:", random_search.best_params_)

# Evaluate model

In [ ]:
# Das History-Objekt abrufen
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

# Anzeigen der Modellzusammenfassung
print(model.summary())

# Plot der Verlustfunktion während des Trainings
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Modell speichern

In [ ]:
model.save('Predict20Days.h5')

# Predictions X-test

In [ ]:
predictions = model.predict(X_test)
print(f'Prediction: {predictions}')
print(len(predictions))
print(len(X_test))

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(predictions[:,-2], color="green", label="Prediction")
plt.plot(y_test[:,-2], color="black", label="Test")
plt.legend()
plt.show()

In [ ]:
# Vorhersagen für das gesamte nächste Jahr (255 Tage) erstellen
predictions = []
input_sequence = X_test[-1]  # Verwende die letzte Sequenz aus den Testdaten
for i in range(20):  # Anzahl der Tage im nächsten Jahr
    # Vorhersage für den nächsten Tag machen
    next_prediction = model.predict(np.expand_dims(input_sequence, axis=0))
    #print(next_prediction)
    predictions.append(next_prediction)
    # Aktualisiere die Eingabesequenz für die nächste Vorhersage
    input_sequence = np.concatenate([input_sequence[1:], next_prediction], axis=0)
    #print(input_sequence)
    
# Vorhersagen in ein numpy array umwandeln
predictions = np.array(predictions)
#print(predictions)

# Output
#print("Shape der Vorhersagen:", predictions.shape)

y_test_trimmed = y_test[:len(predictions)]

for i in range (len(predictions)):
    print(f'Prediction: {predictions[i][0][-2]}', f'Actual: {y_test[i][-2]}')

# Compare test dataset

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(predictions[:,:,-2], color="green", label="Prediction")
plt.plot(y_test_trimmed[:,-2], color="black", label="Test")
plt.legend()
plt.show()

# Prediction for the unkonown sequence

In [ ]:
predictions = model.predict(X_pre)
prediction_20_days_ahead = predictions[0][-2]
print(f'Prediction: {predictions}')
print(len(predictions))
print(len(X_pre))
print(prediction_20_days_ahead)

In [ ]:
X_pre, y_pre = nfPre[:nfPre_size], nfPre[:nfPre_size]
y_pre_tensor = tf.convert_to_tensor(y_pre, dtype=tf.float32)
# Zeitindex anpassen
time_index_shifted = np.arange(20, len(predictions) + 20)

# Plot
plt.figure(figsize=(16, 8))
plt.plot(time_index_shifted, predictions[:, -2], color="green", label="Shifted Prediction")
plt.plot(y_pre[:, 3], color="black", label="Test")
plt.legend()
plt.show()

# Transform back into monetary values

In [ ]:
# Inverse transformation of the predictions
predictions_original_scale = scaler.inverse_transform(predictions)
print(predictions_original_scale)
prediction_20_days_ahead = predictions_original_scale[0][-2]

# shape a 1-Dimensional array
predictions_close = predictions_original_scale[:, -1]  # Nur die Close-Werte extrahieren
#print(predictions_close)

# Show values of inverse transformation
print("Zurücktransformierte Vorhersagen:")
print(predictions_original_scale)
print(prediction_20_days_ahead)

# read test CSV from 2019

In [ ]:
# Benutzerdefinierten Zeitraum und Symbole eingeben
stock_symbols = ['BNR.VI']
start_date = '2019-01-01'
end_date = '2019-01-31'
output_file = 'stock_data_2019.csv'

# Herunterladen und Speichern der Aktiendaten
download_stock_data(stock_symbols, start_date, end_date, output_file)

# CSV-Datei nach Leerzeilen durchsuchen und entfernen
with open(output_file, 'r') as file:
    lines = file.readlines()

# Filtern der Leerzeilen aus den Zeilen
lines = [line.strip() for line in lines if line.strip()]

# Überschreiben der Datei mit den bereinigten Zeilen
with open(output_file, 'w') as file:
    file.write('\n'.join(lines))

In [ ]:
# no normalization. The data is compared to the prediction data from the LSTM
# load CSV
df2019 = pd.read_csv('stock_data_2019.csv')

# transform date to index
df2019.set_index('Date', inplace=True)

print(df2019.head())

# Select columns "Date" and "Close"
selected_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
df2019 = df2019[selected_columns]

# Show first rows
print(df2019.head())

# Compare predictions to actual share values

In [ ]:
# Extrahiere die letzte Zeile
last_row = df2019.iloc[-1]
# Extrahiere den Wert der Spalte "Close" aus der letzten Zeile
actual_close = last_row['Close']
# Erstelle ein DataFrame mit nur einem Eintrag für den tatsächlichen Schlusskurs
comparison_df = pd.DataFrame({'Actual': [actual_close], 'Predicted': prediction_20_days_ahead}, index=[last_row.name])
print(len(comparison_df))
print(comparison_df)


In [ ]:
# reduce quantity of predictions to the length of actual values
predictions_trimmed = predictions_original_scale[:]
print(len(predictions_trimmed))

# reduce quantity of predictions to the length of actual values
df2019_trimmed = df2019[:len(predictions_trimmed)]
print(len(df2019_trimmed))

# Create a data frame with the actual values and the predictions
comparison_df = pd.DataFrame({'Actual': df2019_trimmed['Close'].values, 'Predicted': predictions_trimmed[:,-2]}, index=df2019_trimmed.index)
print(len(comparison_df))
print(comparison_df)

# plot the data
plt.figure(figsize=(10, 6))
plt.plot(comparison_df.index, comparison_df['Actual'], label='Wirklich')
plt.plot(comparison_df.index, comparison_df['Predicted'], label='Vorhersage')
plt.plot(prediction_20_days_ahead, color="red", marker='o', label="20")
plt.xlabel('Datum')
plt.ylabel('Schlusskurs')
plt.title('Vergleich von wirklichen Werten und Vorhersagen')
plt.legend()
plt.grid(True)
plt.show()